GLUE sets: model will be trained on eval set, so you shouldn't also test on the eval set. The problem is that the labels are withheld for the test set. 
Start with SNLI. MultiNLI is a later option too. As is rotten_tomatoes. 
* Victim model performance on dataset train, valid, test set. (done, written code to measure it)
* Create new paraphrased valid + test datasets (done a preliminary version on the valid set) 
* Measure victim model performance on paraphrased datasets (done. on vanilla valid set is about 87% accuracy. generating 16 paraphrases (i.e. not many) and evaluating performance on all of them, we get ~75% accuracy)
* Get document embeddings of original and paraphrased and compare (done)
  * https://github.com/UKPLab/sentence-transformers
* Write a simple way to measure paraphrase quality (done) 
* Construct reward function 


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch 
from torch.utils.data import DataLoader
from datasets import load_dataset, load_metric
import datasets, transformers
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, AutoTokenizer
from pprint import pprint
import numpy as np, pandas as pd
from utils import *   # local script 
import pyarrow
from sentence_transformers import SentenceTransformer, util
from IPython.core.debugger import set_trace
from GPUtil import showUtilization
import seaborn as sns

path_cache = './cache/'
path_results = "./results/"

seed = 420
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
devicenum = torch.cuda.current_device() if device.type == 'cuda' else -1
n_wkrs = 4 * torch.cuda.device_count()
batch_size = 64
pd.set_option("display.max_colwidth", 400)

In [3]:
# Paraphrase model (para)
para_name = "tuner007/pegasus_paraphrase"
para_tokenizer = AutoTokenizer.from_pretrained(para_name)
para_model = AutoModelForSeq2SeqLM.from_pretrained(para_name).to(device)

In [4]:
# Victim Model (VM)
vm_name = "textattack/distilbert-base-cased-snli"
vm_tokenizer = AutoTokenizer.from_pretrained(vm_name)
vm_model = AutoModelForSequenceClassification.from_pretrained(vm_name).to(device)
vm_idx2lbl = vm_model.config.id2label
vm_lbl2idx = vm_model.config.label2id
vm_num_labels = vm_model.num_labels

In [5]:
# Semantic Similarity model 
embedding_model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [6]:
dataset = load_dataset("snli")
train,valid,test = dataset['train'],dataset['validation'],dataset['test']

label_cname = 'label'
remove_minus1_labels = lambda x: x[label_cname] != -1
train = train.filter(remove_minus1_labels)
valid = valid.filter(remove_minus1_labels)
test = test.filter(remove_minus1_labels)

# make sure that all datasets have the same number of labels as what the victim model predicts
assert train.features[label_cname].num_classes == vm_num_labels
assert valid.features[label_cname].num_classes == vm_num_labels
assert test.features[ label_cname].num_classes == vm_num_labels

train_dl = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=n_wkrs)
valid_dl = DataLoader(valid, batch_size=batch_size, shuffle=True, num_workers=n_wkrs)
test_dl = DataLoader( test,  batch_size=batch_size, shuffle=True, num_workers=n_wkrs)

Reusing dataset snli (/data/tproth/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


In [7]:
def get_paraphrases(input_text,num_return_sequences,num_beams, num_beam_groups=1,diversity_penalty=0):
    batch = para_tokenizer(input_text,truncation=True,padding='longest', return_tensors="pt").to(device)
    translated = para_model.generate(**batch,num_beams=num_beams, num_return_sequences=num_return_sequences, 
                                   temperature=1.5, num_beam_groups=num_beam_groups, diversity_penalty=diversity_penalty)
    tgt_text = para_tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

def gen_hypothesis_paraphrases(x, n_seed_seqs=32): 
    """keep n_seed_seqs at 4,8,16,32,64 etc"""
    # TODO: figure out how to batch this. 
    if n_seed_seqs % 4 != 0: raise ValueError("keep n_seed_seqs divisible by 4 for now")
    n = n_seed_seqs/2
    #low diversity (ld) paraphrases 
    ld_l = get_paraphrases(x['hypothesis'],num_return_sequences=int(n),
                            num_beams=int(n))
    #high diversity (hd) paraphrases. We can use num_beam_groups and diversity_penalty as hyperparameters. 
    hd_l =  get_paraphrases(x['hypothesis'],num_return_sequences=int(n),
                            num_beams=int(n), num_beam_groups=int(n),diversity_penalty=50002.5)
    l = ld_l + hd_l 
    x['hypothesis_paraphrases'] = l #TODO: change to list(set(l))             
    return x 


In [8]:
n_seed_seqs = 48
fname = path_cache + 'valid_small_'+ str(n_seed_seqs)
if os.path.exists(fname):  # simple caching
    valid_small = datasets.load_from_disk(fname)
else:
    valid_small = valid.shard(20, 0, contiguous=True)
    valid_small = valid_small.map(lambda x: gen_hypothesis_paraphrases(x, n_seed_seqs=n_seed_seqs),
                  batched=False)
    valid_small.save_to_disk(fname)
    
    

In [9]:
# Create a new dataset by repeating all other fields to be same length as number of paraphrases. 
def create_paraphrase_dataset(batch): 
    """Repeat the other fields to be the same length as the number of paraphrases."""    
    n_premises = len(batch['premise'])
    paraphrases,hyp,prem,labels=[],[],[],[]
    d1 = dict()
    def rep_entry(x): return [x for o in range(n_paraphrases)]
    for p,h,l,hp in zip(batch['premise'], batch['hypothesis'], batch['label'], batch['hypothesis_paraphrases']):
        n_paraphrases = len(hp)
        paraphrases +=    hp
        hyp         +=    rep_entry(h)
        prem        +=    rep_entry(p)
        labels      +=    rep_entry(l)
    return {
        'hypothesis': hyp,
        'hypothesis_paraphrases': paraphrases,
        'premise':prem,
        'label':labels 
       }

In [10]:
fname = path_cache + 'valid_small_paraphrases_' + str(n_seed_seqs)
if os.path.exists(fname):     
    valid_small_paraphrases = datasets.load_from_disk(fname)
else:
    # Need to call this with batched=True to work. 
    valid_small_paraphrases = valid_small.map(create_paraphrase_dataset,batched=True)
    valid_small_paraphrases.save_to_disk(fname)


In [23]:
def get_vm_scores(): 
    """very hacky procedure to generate victim model scores """
    # Get preds and accuracy on the paraphrase dataset
    print("Getting victim model scores.")
    some_dl = DataLoader(valid_small_paraphrases, batch_size=batch_size, shuffle=False, 
                         num_workers=n_wkrs, pin_memory=True)
    dl = some_dl
    metric = load_metric('accuracy')
    para_probs_l,orig_probs_l = [], []
    assert vm_model.training == False  # checks that model is in eval mode 
    #monitor = Monitor(2)  # track GPU usage and memory
    with torch.no_grad():
        for i, data in enumerate(dl): 
            if i % 50 == 0 : print(i, "out of", len(dl))
            labels,premise = data['label'].to(device),data["premise"]
            paraphrases,orig = data["hypothesis_paraphrases"],data["hypothesis"]

            # predictions for paraphrases
            inputs = vm_tokenizer(premise,paraphrases, padding=True,truncation=True, return_tensors="pt")
            inputs.to(device)
            outputs = vm_model(**inputs, labels=labels)
            probs = outputs.logits.softmax(1)
            preds = probs.argmax(1)
            para_probs_l.append(probs.cpu())
            metric.add_batch(predictions=preds, references=labels)

            # predictions for original
            inputs = vm_tokenizer(premise,orig,padding=True,truncation=True, return_tensors="pt")
            inputs.to(device)
            outputs = vm_model(**inputs, labels=labels)
            probs = outputs.logits.softmax(1)
            preds = probs.argmax(1)
            orig_probs_l.append(probs.cpu())        
    para_probs_t,orig_probs_t = torch.cat(para_probs_l),torch.cat(orig_probs_l)
    #monitor.stop()

    # bit of a hack, i'm sure there's a native pytorch function for this but I couldn't find it
    vm_para_scores = torch.tensor([r[idx] for idx,r in zip(valid_small_paraphrases['label'],para_probs_t)])
    vm_orig_scores = torch.tensor([r[idx] for idx,r in zip(valid_small_paraphrases['label'],orig_probs_t)])
    vm_score_decrease = vm_para_scores - vm_orig_scores
    return vm_para_scores,vm_orig_scores,vm_score_decrease

def generate_sim_scores(): 
    """Function to just loop and generate sim scores for each input"""
    print("Getting similarity scores")
    sim_score_l = []
    for i, data in enumerate(valid_small): 
        if i % 50 == 0 : print(i, "out of", len(valid_small))
        orig, para = data['hypothesis'], data['hypothesis_paraphrases']
        orig_emb,para_emb  = embedding_model.encode(orig),embedding_model.encode(para)
        cos_sim = util.cos_sim(orig_emb,para_emb)[0]
        sim_score_l.append(cos_sim)
    sim_score_t = torch.cat(sim_score_l)
    return sim_score_t


In [28]:
# Generate results dataframe 
fname = path_cache + 'results_df' + str(n_seed_seqs) + ".csv"
if os.path.exists(fname):
    results_df = pd.read_csv(fname)
else: 
    vm_para_scores,vm_orig_scores,vm_score_decrease = get_vm_scores()
    sim_score_t = generate_sim_scores()
    results_df = pd.DataFrame({'premise': valid_small_paraphrases['premise'],
                  'orig': valid_small_paraphrases['hypothesis'],
                  'para': valid_small_paraphrases['hypothesis_paraphrases'],
                  'sim_score': sim_score_t,
                  'label': valid_small_paraphrases['label'],
                  'vm_orig_score': vm_orig_scores, 
                  'vm_para_score': vm_para_scores,
                  'vm_change': vm_score_decrease})
    results_df.to_csv(fname)

0 out of 370
50 out of 370
100 out of 370
150 out of 370
200 out of 370
250 out of 370
300 out of 370
350 out of 370
0 out of 493
50 out of 493
100 out of 493
150 out of 493
200 out of 493
250 out of 493
300 out of 493
350 out of 493
400 out of 493
450 out of 493


In [34]:
results_df['vm_change_X_sim_score'] = results_df['vm_change'] * results_df['sim_score']

In [37]:
tmp = results_df.sort_values('vm_change_X_sim_score', ascending=True)

In [38]:
tmp[0:40]

,premise,orig,para,sim_score,label,vm_orig_score,vm_para_score,vm_change,vm_change_X_sim_score
15191,A man with a beard skateboarding and a boy with a blue and black backpack riding a green bike in the background.,There is a man with a gray beard skateboarding.,A man with a beard is skateboarding.,0.916057,1,0.985341,0.006715,-0.978626,-0.896477
23148,A little baby with dirty fingers and smudges on her face points to her blue eye.,A baby points at her green eye.,A baby points at her eye.,0.918273,2,0.978069,0.005282,-0.972787,-0.893284
2323,Kids are on a amusement ride.,Kids ride their favorite amusement ride.,Kids ride an amusement ride.,0.907129,1,0.993661,0.010853,-0.982808,-0.891534
23143,A little baby with dirty fingers and smudges on her face points to her blue eye.,A baby points at her green eye.,A baby points to her eye.,0.899644,2,0.978069,0.003546,-0.974523,-0.876724
23159,A little baby with dirty fingers and smudges on her face points to her blue eye.,A baby points at her green eye.,A baby points towards her eye.,0.897910,2,0.978069,0.007429,-0.970640,-0.871548
15181,A man with a beard skateboarding and a boy with a blue and black backpack riding a green bike in the background.,There is a man with a gray beard skateboarding.,A man with a beard is on a skateboard.,0.891820,1,0.985341,0.010389,-0.974952,-0.869482
10770,An asian woman sitting outside an outdoor market stall.,A woman sitting in an indoor market.,A woman sitting in a market.,0.890446,2,0.992062,0.015992,-0.976070,-0.869137
784,"Under a blue sky with white clouds, a child reaches up to touch the propeller of a plane standing parked on a field of grass.",A child is reaching to touch the propeller out of curiosity.,A child is reaching to touch the propeller.,0.914637,1,0.975338,0.025472,-0.949867,-0.868783
791,"Under a blue sky with white clouds, a child reaches up to touch the propeller of a plane standing parked on a field of grass.",A child is reaching to touch the propeller out of curiosity.,A child is reaching out to touch a propeller.,0.908909,1,0.975338,0.020866,-0.954472,-0.867528
792,"Under a blue sky with white clouds, a child reaches up to touch the propeller of a plane standing parked on a field of grass.",A child is reaching to touch the propeller out of curiosity.,A child is reaching to touch a propeller.,0.906859,1,0.975338,0.019427,-0.955911,-0.866876


In [46]:
results_df.iloc[9456:]

,premise,orig,para,sim_score,label,vm_orig_score,vm_para_score,vm_change,vm_change_X_sim_score
9456,A lady with bright orange hair walking in a crowd.,The woman is asleep.,The woman is sleepy.,0.875720,2,0.998881,0.235266,-0.763615,-0.668713
9457,A lady with bright orange hair walking in a crowd.,The woman is asleep.,The woman is not awake.,0.788796,2,0.998881,0.024462,-0.974419,-0.768618
9458,A lady with bright orange hair walking in a crowd.,The woman is asleep.,The woman is sleeping.,0.948761,2,0.998881,0.999148,0.000267,0.000253
9459,A lady with bright orange hair walking in a crowd.,The woman is asleep.,The woman is asleep.,1.000000,2,0.998881,0.998881,0.000000,0.000000
9460,A lady with bright orange hair walking in a crowd.,The woman is asleep.,A woman is sleeping.,0.936337,2,0.998881,0.999266,0.000385,0.000360
...,...,...,...,...,...,...,...,...,...
23659,"Nine women in white robes with hoods walk on plush, green grass.",The women are wearing flip flops.,These Women have flipped flop shoes on,0.726975,1,0.100099,0.222727,0.122628,0.089147
23660,"Nine women in white robes with hoods walk on plush, green grass.",The women are wearing flip flops.,Woman withflops,0.678554,1,0.100099,0.112493,0.012394,0.008410
23661,"Nine women in white robes with hoods walk on plush, green grass.",The women are wearing flip flops.,Woman withflops on them,0.691026,1,0.100099,0.168826,0.068727,0.047492
23662,"Nine women in white robes with hoods walk on plush, green grass.",The women are wearing flip flops.,Three girls were dressed inflip flops,0.525657,1,0.100099,0.147149,0.047051,0.024732


In [31]:
grouped_df = results_df.groupby(['premise', 'orig'])

In [33]:
results_df.sort_values()

TypeError: sort_values() missing 1 required positional argument: 'by'

In [20]:
results_df.iloc[2000:2040,:]

,premise,orig,para,sim_score,label,vm_orig_score,vm_para_score,vm_change
2000,"At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.",A crowd is in front of a camera,the people are standing in front of the cameras,0.647868,1,0.022584,0.083856,0.061272
2001,"At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.",A crowd is in front of a camera,While a crowd is standing,0.627243,1,0.022584,0.093733,0.071148
2002,"At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.",A crowd is in front of a camera,people in front of a camera.,0.743037,1,0.022584,0.015049,-0.007535
2003,"At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.",A crowd is in front of a camera,There is a crowd.. and a camera,0.875868,1,0.022584,0.013921,-0.008664
2004,"At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.",A crowd is in front of a camera,While a crowd is standing together,0.610570,1,0.022584,0.091407,0.068822
2005,"At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.",A crowd is in front of a camera,people in front of a camera. s,0.706776,1,0.022584,0.025972,0.003387
2006,"At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.",A crowd is in front of a camera,Crowd standing near camera,0.772629,1,0.022584,0.196660,0.174076
2007,"At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.",A crowd is in front of a camera,Everyone except for the camera is standing around,0.519551,1,0.022584,0.342093,0.319509
2008,"At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.",A crowd is in front of a camera,Crowd standing near camera with one behind them,0.706284,1,0.022584,0.225278,0.202693
2009,"At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.",A crowd is in front of a camera,Some of the group is near the cameras,0.605389,1,0.022584,0.144014,0.121429


## Archive 

In [ ]:
# # calculates performance of victim model on a dataloader 
# dl = valid_dl
# metric = load_metric('accuracy')
# for i, data in enumerate(dl): 
#     if i % 10 == 0 : print(i, "out of", len(dl)) 
#     labels,premise,hypothesis = data['label'].to(device),data["premise"],data["hypothesis"]
#     inputs = vm_tokenizer(premise,hypothesis, padding=True,truncation=True, return_tensors="pt")
#     inputs.to(device)
#     outputs = vm_model(**inputs, labels=labels)
#     probs = outputs.logits.softmax(1)
#     preds = probs.argmax(1)
#     metric.add_batch(predictions=preds, references=labels)

# metric.compute()


In [ ]:
# # Score semantic similarity with cross encoders
# from sentence_transformers.cross_encoder import CrossEncoder
# cross_encoder= CrossEncoder('cross-encoder/quora-distilroberta-base')
# i =11
# data = valid_small[i]
# orig, para = data['hypothesis'], data['hypothesis_paraphrases']
# orig_rep = [orig for i in range(len(para))]
# pairs = list(zip(orig_rep,para))
# scores = cross_encoder.predict(pairs)
# results_df = pd.DataFrame({'pairs':pairs, 'para': para,'score': cos_sim})
# print(orig)
# results_df.sort_values('score', ascending=False)

In [14]:
# # with sentence transformers
# valid_small_dl = DataLoader(valid_small, batch_size=4, shuffle=False, 
#                      num_workers=n_wkrs, pin_memory=True)
# sim_score_l = []
# for i, data in enumerate(valid_small_dl): 
#     pass
#     orig, para = data['hypothesis'], data['hypothesis_paraphrases']
#     orig_emb,para_emb  = embedding_model.encode(orig),embedding_model.encode(para)
# #     cos_sim = util.cos_sim(orig_emb,para_emb)[0]
# #     results_df = pd.DataFrame({'para': para,'score': cos_sim})
# #     print(orig)
# #     results_df.sort_values('score', ascending=False)

IndentationError: unexpected indent (<ipython-input-14-4e07410ec837>, line 7)